#### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split

import xgboost as xgb

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, rand
from joblib import Parallel, delayed
import multiprocessing

#### Import the dataset

In [ ]:
breast_cancer_X, breast_cancer_y = load_breast_cancer(return_X_y = True)
X = pd.DataFrame(breast_cancer_X)
y = pd.Series(breast_cancer_y).map({0:1, 1:0})

X.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Define the hyperparameter space

In [ ]:
param_grid = {
    'n_estimators': hp.quniform('n_estimators', 200, 2500, 100),
    'max_depth': hp.uniform('max_depth', 1, 10),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.99),
    'booster': hp.choice('booster', ['gbtree', 'dart']),
    'gamma': hp.quniform('gamma', 0.01, 10, 0.1),
    'subsample': hp.uniform('subsample', 0.50, 0.90),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.50, 0.99),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.50, 0.99),
    'colsample_bynode': hp.uniform('colsample_bynode', 0.50, 0.99),
    'reg_lambda': hp.uniform('reg_lambda', 1, 20)
}

#### Define the objective function

In [ ]:
def objective(params):
    params_dict = {
        'n_estimators': int(params['n_estimators']),
        'max_depth': int(params['max_depth']),
        'learning_rate': params['learning_rate'],
        'booster': params['booster'],
        'gamma': params['gamma'],
        'subsample': params['subsample'],
        'colsample_bytree': params['colsample_bytree'],
        'colsample_bylevel': params['colsample_bylevel'],
        'colsample_bynode': params['colsample_bynode'],
        'random_state': 1000,
    }

    # Criando o modelo com os parâmetros
    gbm = xgb.XGBClassifier(**params_dict)

    # Avaliando o modelo com validação cruzada
    score = cross_val_score(gbm, X_train, y_train, scoring='accuracy', cv=5, n_jobs=-1).mean()

    # Retornando a pontuação negativa porque Hyperopt minimiza a função objetivo
    return {'loss': -score, 'status': STATUS_OK}

#### Randomized Search

In [ ]:
search = fmin(
    fn=objective,
    space=param_grid,
    max_evals=50,
    rstate=np.random.default_rng(42),
    algo=rand.suggest,
)

 46%|████▌     | 23/50 [1:59:58<5:19:41, 710.41s/trial, best loss: -0.9723101265822784]

In [ ]:
search

In [ ]:
best_hp_dict = {
                'n_estimators': int(search['n_estimators']),
                'max_depth': int(search['max_depth']),
                'learning_rate': search['learning_rate'],
                'booster': search['booster'],
                'gamma': search['gamma'],
                'subsample': search['subsample'],
                'colsample_bytree': search['colsample_bytree'],
                'colsample_bylevel': search['colsample_bylevel'],
                'colsample_bynode': search['colsample_bynode'],
                'random_state': 1000,
                }

In [ ]:
gbm_final = xbg.XGBClassifier(**best_hp_dict)
gbm_final.fit(X_train, y_train)

In [ ]:
X_train_preds = gbm_final.predict(X_train)
X_test_preds = gbm_final.predict(X_test)

print('Train roc_auc: ', accuracy_score(y_train, X_train_preds))
print('Test roc_auc: ', accuracy_score(y_test, X_test_preds))

#### Evaluating the search

In [ ]:
trials = Trials()

In [ ]:
second_search = fmin(
    fn=objective,
    space=param_grid,
    max_evals=50,
    rstate=np.random.default_rng(42),
    algo=rand.suggest,
    trials=trials
)

In [ ]:
second_search

In [ ]:
pd.DataFrame(trials.vals).head()

In [ ]:
pd.DataFrame(trials.results).head()

In [ ]:
results = pd.concat([
    pd.DataFrame(trials.vals),
    pd.DataFrame(trials.results)],
                    axis=1,
                    ).sort_values(by='loss', ascending=False).reset_index(drop=True)

results.head()

In [ ]:
results['loss'].plot()
plt.ylabel('Accuracy')
plt.xlabel('Hyperparam combination')

In [ ]:
pd.DataFrame(trials.results)['loss'].min()